In [9]:
%load_ext autoreload
%autoreload 2

import duckdb
import json
import os
import pandas as pd
import pathlib

from utils import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
# Initialize DuckDB connection
conn = duckdb.connect("Piper.db")

In [11]:
conn.execute("DROP TABLE IF EXISTS especies_nativas_endemicas")
conn.execute("""
    CREATE TABLE especies_nativas_endemicas AS
    SELECT * FROM especies_nativas
    UNION ALL
    SELECT * FROM especies_endemicas
""")

In [12]:
df = conn.execute("""
    SELECT scientificName_y, redListCategory, COUNT(DISTINCT(land_regions)) AS region_count
    FROM especies_nativas_endemicas
    GROUP BY scientificName_y, redListCategory
    ORDER BY region_count DESC
""").df()
df.head()

,scientificName_y,redlistCategory,region_count
0,Piper amalago,Least Concern,35
1,Piper aduncum,Least Concern,32
2,Piper hispidum,Least Concern,27
3,Piper tuberculatum,Least Concern,26
4,Piper dilatatum,Least Concern,25


In [13]:
conn.execute("SELECT DISTINCT(redlistCategory) FROM especies_nativas_endemicas").df()

,redlistCategory
0,Least Concern
1,Critically Endangered
2,Near Threatened
3,Vulnerable
4,Endangered


In [14]:
df["enumRedListCategory"] = df["redlistCategory"].map({
    "Least Concern": 0,
    "Near Threatened": 0,
    "Critically Endangered": 1,
    "Endangered": 1,
    "Vulnerable": 1
})

In [15]:
df.to_csv("regression.csv", sep=";", quoting=2, index=False, encoding="utf-8")

In [16]:
conn.close()